# Final Project: Data Parsing, Data Acquisition, and Data Storage
## Calvin, Jack

## Data:
1. Climate_Data.xml: This is data on the Global temperature Anomalies compared to the set average from 1901 - 2010 
- NOAA National Centers for Environmental Information, Climate at a Glance: Global Time Series, published March 2023, retrieved on March 29, 2023, from https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/global/time-series
- We know this data is available since we were given the option to download right on the webiste ... it is available because of the Freedom of Information Act. https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/global/time-series
2. Disaster_Data.html(broken up into the different decades): Data on the decade average deaths per country for different types of natural disasters.
- Hannah Ritchie, Pablo Rosado and Max Roser (2022) - "Natural Disasters". Published online at OurWorldInData.org. Retrieved from: 'https://ourworldindata.org/natural-disasters' 
- This Data is free for us to use since we arent connecting to the website ... "All visualizations, data, and code produced by Our World in Data are completely open access under the Creative Commons BY license"


## Central Questions:
1. Have natural disasters become worse over the decades as Global warming has increased?
- Measured by comparing the number of deaths from natural disasters by Decade

2. Which natural disasters have grown the most in severity? 
- Measured by the number of deaths over time

## How we plan to use the data
Using this data we hope to gather concrete answers for our two central questions. For our first central questing We plan to clean up our data in order to make a comparison between how far the temperature anomalies from the climate data and how many deaths there have been over the decades. For second question inorder to answer we first need to break up our natural disaster data into overall deaths per the different disaster. Once we have it organized in this way we can compare the amount of deaths from the start of data collection to the end and see which natural disasters death rates have grown the most.

# Context to the Questions
It seems like everyday there is a new article or news story about how climate change is getting worse. Just recently the Willow project in Alaska was approved by the Biden administration. The Goal of this project is to see just how much worse it is getting. Using our data we want to see how many more people are dying from natural disasters today compared to in 1900. We obviously need to first see if there is a connection between climate change and natural disaster severity. This could give a quantifyable answer to one part of the question.

Willow project approval: https://www.npr.org/2023/03/13/1163075377/willow-drilling-project-alaska-approved-biden


### Code Libraries

In [21]:
from lxml import etree
from lxml import html
import os.path as osp
import pandas as pd
import importlib
import sqlalchemy as sa
import io
import sys
import os
import json

datadir = "Data"
htmlparser =  etree.HTMLParser()


# Deliverable: 2
## Data Acquisition

### Code_Segment: 1
Aquiring the root for the Climate data(Climate_Data.xml)

In [22]:
Climate_path = osp.join(datadir, "Climate_Data.xml")
parser = etree.XMLParser(remove_blank_text = True)
tree = etree.parse(Climate_path, parser)

Climate_root = tree.getroot()

Climate_root

<Element dataCollection at 0x7f8a2120c440>

### Code_Segment: 2
Pulling the data from Climate_root in order to make a pandas data frame

In [23]:
Climate_data = []

"""Iterate through Climate_xml and extract the data"""
for child in Climate_root:
    item = {}
    for subchild in child:
        item[subchild.tag] = subchild.text
    Climate_data.append(item)

"""Create a Pandas dataframe from the data"""
Climate_DF = pd.DataFrame(Climate_data)

"""Print the dataframe"""
Climate_DF

,title,units,basePeriod,missing,year,value
0,"Global Land and Ocean Temperature Anomalies, J...",Degrees Celsius,1901-2000,-999,NaN,NaN
1,NaN,NaN,NaN,NaN,1850,-0.17
2,NaN,NaN,NaN,NaN,1851,-0.09
3,NaN,NaN,NaN,NaN,1852,-0.10
4,NaN,NaN,NaN,NaN,1853,-0.12
...,...,...,...,...,...,...
169,NaN,NaN,NaN,NaN,2018,0.86
170,NaN,NaN,NaN,NaN,2019,0.97
171,NaN,NaN,NaN,NaN,2020,1.01
172,NaN,NaN,NaN,NaN,2021,0.86


### Code_Segment: 3
Light Cleaning of "Climate_DF"

In [24]:
"""Dropping all unnecessary Columns"""
Climate_DF.drop(["title", "units", "basePeriod", "missing"], axis = 1, inplace = True)

"""Dropping Rows with NaN Values"""
Climate_DF.dropna(inplace = True)

Climate_DF["Anomaly"] = Climate_DF["value"]
Climate_DF.drop(["value"], axis = 1, inplace = True)

Climate_DF

,year,Anomaly
1,1850,-0.17
2,1851,-0.09
3,1852,-0.10
4,1853,-0.12
5,1854,-0.11
...,...,...
169,2018,0.86
170,2019,0.97
171,2020,1.01
172,2021,0.86


## Data Details: Global Climate Data


### Code_Segment: 4
Creating all of the paths for the Disaster_(decade).html's

In [25]:
"""Create an empty list to hold the file paths"""
Disaster_paths = []

"""Set up the list of file names"""
file_names = ['Disaster_1900.html', 'Disaster_1910.html', 'Disaster_1920.html', 'Disaster_1930.html', 
              'Disaster_1940.html', 'Disaster_1950.html', 'Disaster_1960.html', 'Disaster_1970.html', 
              'Disaster_1980.html', 'Disaster_1990.html', 'Disaster_2000.html', 'Disaster_2010.html', 
              'Disaster_2020.html']


"""Use os.path.join() to join the directory path and each file name"""
for filename in file_names:
    file_path = os.path.join(datadir, filename)
    Disaster_paths.append(file_path)

"""Print the list of file paths"""
print(Disaster_paths)

['Data/Disaster_1900.html', 'Data/Disaster_1910.html', 'Data/Disaster_1920.html', 'Data/Disaster_1930.html', 'Data/Disaster_1940.html', 'Data/Disaster_1950.html', 'Data/Disaster_1960.html', 'Data/Disaster_1970.html', 'Data/Disaster_1980.html', 'Data/Disaster_1990.html', 'Data/Disaster_2000.html', 'Data/Disaster_2010.html', 'Data/Disaster_2020.html']


### Code_Segment: 5
Parsing and gathering the roots of the parsed "Disaster_paths" list

In [26]:

Disaster_roots = []

"""Iterate through the list of file paths, parse each file, and extract its root element"""
for path in Disaster_paths:
    """Open the HTML file and read its contents into a string"""
    with open(path, 'r') as f:
        html_string = f.read()

    """Parse the HTML string using the lxml HTML parser"""
    html_tree = etree.parse(io.StringIO(html_string), etree.HTMLParser())

    """Get the root of the HTML tree"""
    Disaster_roots.append(html_tree.getroot())

    """Print the root element"""

print(Disaster_roots)

[<Element html at 0x7f8a2051d9c0>, <Element html at 0x7f8a112e3ac0>, <Element html at 0x7f8a207c3d80>, <Element html at 0x7f8a21190e40>, <Element html at 0x7f8a21190d80>, <Element html at 0x7f8a21469f40>, <Element html at 0x7f8a211b6540>, <Element html at 0x7f8a211b6400>, <Element html at 0x7f8a211b6880>, <Element html at 0x7f8a211b6480>, <Element html at 0x7f8a211b6180>, <Element html at 0x7f8a211b6700>, <Element html at 0x7f8a215b8500>]


### Code_Segment: 6
Finding the Table node for each root in "Disaster_roots" 

In [27]:
"""Create an empty list"""
Disaster_tableroots = []

"""Iterate through each root in the list Disaster_roots, locate the tablenode using xpath then get the tableroot from that. Append tableroot to the list"""
for root in Disaster_roots:
    tableroot = root.xpath("/html/body/main/article/div[2]/div[2]/div/div/section[1]/figure/div/div[3]/div/div[1]/div/table")
    Disaster_tableroots.append(tableroot)

"""print the Disaster table roots"""
print(Disaster_tableroots)

[[<Element table at 0x7f8a215b3e80>], [<Element table at 0x7f8a10ff4dc0>], [<Element table at 0x7f8a2119b280>], [<Element table at 0x7f8a2119b380>], [<Element table at 0x7f8a2119b240>], [<Element table at 0x7f8a211b0540>], [<Element table at 0x7f8a1123ca00>], [<Element table at 0x7f8a2119b340>], [<Element table at 0x7f8a211b0d40>], [<Element table at 0x7f8a211b0e00>], [<Element table at 0x7f8a215b72c0>], [<Element table at 0x7f8a2119b400>], [<Element table at 0x7f8a2079b040>]]


### Code_Segment: 7
Quick check to make sure each table root is the correct root

In [28]:
"""iterates through each tableroot in Disaster_tableroots and checks the class"""
for table in Disaster_tableroots:
    assert table[0].get("class") == "data-table"

### Code_Segment: 8
Figure out how to get the columns from the html

In [29]:
"""uses the given path to collect the country column name"""
country_col = Disaster_tableroots[0][0].xpath(".//thead/tr/th/div/span/text()")
print(country_col)

"""uses the given path to collect the different weather column names"""
weather_col = Disaster_tableroots[0][0].xpath(".//thead/tr/th/div/span/div[@class = 'name']/text()")
print(weather_col)

['Country']
['Fog', 'Glacial lake outbursts', 'Wildfires', 'Dry mass movements', 'Landslides', 'Volcanic activity', 'Extreme temperatures', 'Extreme weather', 'Earthquakes', 'Floods', 'Drought']


### Code_Segment: 9
Figure out how to get the data rows from the html

In [30]:
country_row = Disaster_tableroots[0][0].xpath(".//td[position() = 1]/text()")

for i in range(13):
    Fog_row = Disaster_tableroots[i][0].xpath(".//td[position() = 2]/span/text()")
    Glaciallakeoutbursts_row = Disaster_tableroots[i][0].xpath(".//td[position() = 3]/span/text()")
    Wildfires_row = Disaster_tableroots[i][0].xpath(".//td[position() = 4]/span/text()")
    Drymassmovements_row = Disaster_tableroots[i][0].xpath(".//td[position() = 5]/span/text()")
    Landslides_row = Disaster_tableroots[i][0].xpath(".//td[position() = 6]/span/text()")
    Volcanicactivity_row = Disaster_tableroots[i][0].xpath(".//td[position() = 7]/span/text()")
    Extremetemperatures_row = Disaster_tableroots[i][0].xpath(".//td[position() = 8]/span/text()")
    Extremeweather_row = Disaster_tableroots[i][0].xpath(".//td[position() = 9]/span/text()")
    Earthquakes_row = Disaster_tableroots[i][0].xpath(".//td[position() = 10]/span/text()")
    Floods_row = Disaster_tableroots[i][0].xpath(".//td[position() = 11]/span/text()")
    Drought_row = Disaster_tableroots[i][0].xpath(".//td[position() = 12]/span/text()")
    
    DoL = {country_col[0] : country_row,weather_col[0]:Fog_row,weather_col[1]:Glaciallakeoutbursts_row,weather_col[2]:Wildfires_row,
               weather_col[3]:Drymassmovements_row,weather_col[4]:Landslides_row,weather_col[5]:Volcanicactivity_row,weather_col[6]:Extremetemperatures_row,weather_col[7]:Extremeweather_row,weather_col[8]:Earthquakes_row,
                weather_col[9]:Floods_row,weather_col[10]:Drought_row
               }
    if i == 0:
        Dyear1900 = pd.DataFrame.from_dict(DoL)
    elif i == 1:
        Dyear1910 = pd.DataFrame.from_dict(DoL)
    elif i == 2:
        Dyear1920 = pd.DataFrame.from_dict(DoL)
    elif i == 3:
        Dyear1930 = pd.DataFrame.from_dict(DoL)
    elif i == 4:
        Dyear1940 = pd.DataFrame.from_dict(DoL)
    elif i == 5:
        Dyear1950 = pd.DataFrame.from_dict(DoL)
    elif i == 6:
        Dyear1960 = pd.DataFrame.from_dict(DoL)
    elif i == 7:
        Dyear1970 = pd.DataFrame.from_dict(DoL)
    elif i == 8:
        Dyear1980 = pd.DataFrame.from_dict(DoL)
    elif i == 9:
        Dyear1990 = pd.DataFrame.from_dict(DoL)
    elif i == 10:
        Dyear2000 = pd.DataFrame.from_dict(DoL)
    elif i == 11:
        Dyear2010 = pd.DataFrame.from_dict(DoL)
    elif i == 12:
        Dyear2020 = pd.DataFrame.from_dict(DoL)

### Code_Segment: 10
Create a dataframe from each decades tableroot and add a year column

In [31]:
DF_1900 = Dyear1900
DF_1900["Decade"] = 1900

DF_1910 = Dyear1910
DF_1910["Decade"] = 1910

DF_1920 = Dyear1920
DF_1920["Decade"] = 1920

DF_1930 = Dyear1930
DF_1930["Decade"] = 1930

DF_1940 = Dyear1940
DF_1940["Decade"] = 1940

DF_1950 = Dyear1950
DF_1950["Decade"] = 1950

DF_1960 = Dyear1960
DF_1960["Decade"] = 1960

DF_1970 = Dyear1970
DF_1970["Decade"] = 1970

DF_1980 = Dyear1980
DF_1980["Decade"] = 1980

DF_1990 = Dyear1990
DF_1990["Decade"] = 1990

DF_2000 = Dyear2000
DF_2000["Decade"] = 2000

DF_2010 = Dyear2010
DF_2010["Decade"] = 2010

DF_2020 = Dyear2020
DF_2020["Decade"] = 2020

### Code_Segment: 11
Combine all of the data frames 

In [32]:
Disaster_DF = pd.concat([Dyear1900, Dyear1910, Dyear1920, Dyear1930, Dyear1940 ,Dyear1950, Dyear1960, Dyear1970, Dyear1980, Dyear1990,Dyear2000, Dyear2010, Dyear2020], axis=0, ignore_index=True)
Disaster_DF

,Country,Fog,Glacial lake outbursts,Wildfires,Dry mass movements,Landslides,Volcanic activity,Extreme temperatures,Extreme weather,Earthquakes,Floods,Drought,Decade
0,Afghanistan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1900
1,Africa,0.00,0.00,0.00,0.00,0.00,1.70,0.00,0.00,10.00,0.00,"1,100.00",1900
2,Albania,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,12.00,0.00,0.00,1900
3,Algeria,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1900
4,American Samoa,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1900
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3050,World,0.00,87.33,92.00,0.00,342.00,30.67,"2,481.67","1,690.33","1,408.67","5,916.33",836.67,2020
3051,Yemen,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,67.00,0.00,2020
3052,Yugoslavia,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2020
3053,Zambia,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,0.00,2020


## Deliverable: 3
### Data Cleaning/SQL Database construction

### Code_segement: 12
Organizing our different natural disasters into "natural_disaster" and "deaths" Columns

In [33]:
"""Melting all the different natural disasters into a single natural disaster column and a Deaths column"""
Disaster_DF = Disaster_DF.melt(id_vars = ["Country", "Decade"], value_vars = ["Fog","Glacial lake outbursts", "Wildfires", "Dry mass movements", "Landslides", "Volcanic activity", "Extreme weather", "Extreme temperatures", "Earthquakes", "Floods", "Drought"], var_name = "Natural Disasters", value_name = "Deaths")

### Code_segment: 13
All deaths are averages based on the decade so we need to multiply all values in Deaths by 10

In [34]:
""" Getting rid of the commas inorder to make the values floats"""
Disaster_DF['Deaths'] = Disaster_DF['Deaths'].str.replace(',', '').astype(float)

"""Convert the 'Deaths' column to integer type and multiply by 10, rounded to the nearest integer"""
Disaster_DF['Deaths'] = round(Disaster_DF['Deaths'] * 10).astype(int)

Disaster_DF

,Country,Decade,Natural Disasters,Deaths
0,Afghanistan,1900,Fog,0
1,Africa,1900,Fog,0
2,Albania,1900,Fog,0
3,Algeria,1900,Fog,0
4,American Samoa,1900,Fog,0
...,...,...,...,...
33600,World,2020,Drought,8367
33601,Yemen,2020,Drought,0
33602,Yugoslavia,2020,Drought,0
33603,Zambia,2020,Drought,0


### Code_segment: 14
Quick cleaining of "Climate_DF"

## SQL Database Creation


In [35]:
def getsqlite_creds(dirname=".",filename="creds.json",source="sqlite"):
    """ Using directory and filename parameters, open a credentials file
        and obtain the two parts needed for a connection string to
        a local provider using the "sqlite" dictionary within
        an outer dictionary.  
        
        Return a scheme and a dbfile
    """
    assert osp.isfile(osp.join(dirname, filename))
    with open(osp.join(dirname, filename)) as f:
        D = json.load(f)
    sqlite = D[source]
    return sqlite["scheme"], sqlite["dbdir"], sqlite["database"]


def buildConnectionString(source="sqlite_book"):
    scheme, dbdir, database = getsqlite_creds(source=source)
    template = '{}:///{}/{}.db'
    return template.format(scheme, dbdir, database)

### Code_segment: 15
Building a connection between the new "Weather.db" and our python notebook

In [36]:
"""Build the conection string"""
cstring = buildConnectionString("sqlite_weather")
print("Connection string:", cstring)

"""Connect to the database"""
engine = sa.create_engine(cstring)
connection = engine.connect()

Connection string: sqlite:///./Proj_DB/Weather.db


### Code_segment: 16
Creating a "Climate_DF" table in "Weather.db"

In [37]:
"""Anomalies Table Creation Statement"""
C1_statement = """
    CREATE TABLE IF NOT EXISTS Anomalies (
        year INT NOT NULL,
        Anomaly Float(6) NOT NULL,
        PRIMARY KEY(year))
    """

try:
    connection.execute(C1_statement)
except sa.exc.SQLAlchemyError as err:
    print("CREATE of Anomalies failed:", str(err))

### Code_segment: 17
Creating a "Disaster_DF" table in "Weather.db"

In [38]:
"""Disasters Table Creation Statement"""
C2_statement = """
    CREATE TABLE IF NOT EXISTS Disasters(
        Country VARCHAR(25) NOT NULL,
        Decade INT NOT NULL,
        [Natural Disasters] VARCHAR(20) NOT NULL,
        Deaths INT,
        PRIMARY KEY(Country, Decade, [Natural Disasters])
    )
    """

try:
    connection.execute(C2_statement)
except sa.exc.SQLAlchemyError as err:
    print("CREATE of Disasters failed:", str(err))   

### Code_segment: 19
Fill up our newly created tables

In [39]:
"""Inserting data into Anomalies directly from a DataFrame"""
Climate_DF.to_sql("Anomalies", con = connection, if_exists = "append", index = False)

"""Inserting data into Disasters directly from a DataFrame"""
Disaster_DF.to_sql("Disasters", con = connection, if_exists = "append", index = False)

33605

### Code_segment: 20
Checking our SQL tables

In [40]:
Clim_SQL = pd.read_sql_table("Anomalies", connection)

Clim_SQL

,year,Anomaly
0,1850,-0.17
1,1851,-0.09
2,1852,-0.10
3,1853,-0.12
4,1854,-0.11
...,...,...
168,2018,0.86
169,2019,0.97
170,2020,1.01
171,2021,0.86


In [41]:
NatDis_SQL = pd.read_sql_table("Disasters", connection)
NatDis_SQL

,Country,Decade,Natural Disasters,Deaths
0,Afghanistan,1900,Fog,0
1,Africa,1900,Fog,0
2,Albania,1900,Fog,0
3,Algeria,1900,Fog,0
4,American Samoa,1900,Fog,0
...,...,...,...,...
33600,World,2020,Drought,8367
33601,Yemen,2020,Drought,0
33602,Yugoslavia,2020,Drought,0
33603,Zambia,2020,Drought,0


### Code_segement: 21
Close the connection

In [42]:
"""Close the connection!"""
try:
    connection.close()
except:
    pass
del engine

## Data Base Design:

The Weather Data Base has two tables in it with no relationships between them. This was one of our issues when planning this and now we plan to create a linking decade column with sql queries using outer joins. 

The Anomalies table has two fields year and Anomaly. Within this table no Null values are allowed for either. The year field is has INT values and is the primary key. Anomaly is a float value.

the Disasters table has 4 fields. The Natural Disasters field has VARCHAR values and is one of the primary keys. The Country field has VARCHAR values and is one of the primary keys. The Decade field has int values and is the last field which makes up our Primary Key. The Deaths field is our only non-primary key field and has int values.

## Sound DB Design:
The Weather data base has to adhere to sound database design. For this sound data base design our tables represent entities and both names are plural nouns. Our primary keys within the two different tabels are both NOT NULL and wont change. The key for Anomalies is singleton and the key for Disasters is composite. We have eliminated values as fields as fieldname  and redundant information. Within the Data Base creaion there is no Relationship because Anomalies is based on years and Disasters is based on decades. keys are meaningful and we dont use mashups.